In [8]:
import os
from PIL import Image

In [9]:
X = []
Y = []
width = 20
height = 20 # resize to 100x100 to solve the RAM issue in Kaggle notebooks
INPUT_CLASSES_DIR = 'face_age/'

import numpy as np

for folder_name,_,filenames in os.walk(INPUT_CLASSES_DIR):
    #if folder_name !="face_age" and folder_name != 'Data':
    if folder_name !="face_age":
        for file in filenames:
            file_path = folder_name +"/"+ file
            image = Image.open(file_path).convert('L').resize((width, height))
            X.append(np.array(image).flatten())
            #X.append(np.array(image))
            Y.append(int(folder_name[-3:]))
    else:
        pass

In [10]:
import pandas as pd

lst = list(zip(X, Y))


df = pd.DataFrame(X)
df['age'] = Y
print(df.head())

# df = pd.DataFrame(lst, columns = ['image', 'age'])

# df.head()



     0    1    2    3    4    5    6    7   8    9  ...  391  392  393  394  \
0  127 -115 -108 -103  -98  -94  -90  -87 -84  -78  ...  125  -75  -63  -78   
1  -19  -50  -51  -46  -54  -58  -62  -63 -63  -60  ... -105 -112 -121  111   
2  -56  121   77   80  -90  -54  -43  -41 -40  -41  ...   68   84   85   94   
3 -112 -109 -101  -96 -100 -109 -121 -112 -92 -104  ...    0    0    0    0   
4   86 -124  -67  -62  -55  -52  -53  -56 -59  -63  ...  -55  -53  -46  -32   

   395  396  397  398  399  age  
0  -88  -73  -55  -53  -46    1  
1  108 -128   88  103 -120    1  
2  101  104  110  117  118    1  
3    0    0    0    0    0    1  
4  -34  -36  -46  -50  -42    1  

[5 rows x 401 columns]


In [11]:
# first = df.iloc[0]

# # each image is a 100x100 pixel 2D array where each pixel is an array of 3 values in form of RGB
# first['image'].shape

In [12]:
#df.to_csv('dataframe.csv')

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from scipy.stats import norm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
import math


def plot_effect_of_C(data, hp1, hp2, hpv1, hpv2):

    means = []
    Cs = []

    for m, s, p in data:
        if p[hp1] == hpv1 and p[hp2] == hpv2:
            means.append(m)
            Cs.append(p['C'])

    plt.plot(range(len(Cs)),means)
    plt.xticks(range(len(Cs)), Cs)
    plt.title('hyperparameter vs accuracy')
    plt.xlabel('hyperparameter C')
    plt.ylabel('accuracy')
    plt.show()

def train_test_SVM(features, target):
    # X_train, X_test, y_train, y_test = train_test_split(features, target)
    
    # kernel = ['poly', 'rbf', 'sigmoid']
    # c_values = [50, 10, 1.0, 0.1, 0.01]
    # gamma = ['scale', 'auto']

    # grid = dict(kernel=kernel,C=c_values,gamma=gamma)
    # cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # grid_search = GridSearchCV(estimator=SVC(), param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)

    # grid_result = grid_search.fit(X_train, y_train)
    
    # print("Best params", grid_result.best_params_)
    # means = grid_result.cv_results_['mean_test_score']
    # stds = grid_result.cv_results_['std_test_score']
    # params = grid_result.cv_results_['params']

    # for mean, stdev, param in zip(means, stds, params):
    #     print("%f (%f) with: %r" % (mean, stdev, param))


    # plot_effect_of_C(zip(means, stds, params), 'gamma', 'kernel', grid_result.best_params_['gamma'], grid_result.best_params_['kernel'])

    # print(target.head())
    # print(features.head())
        
    data_rows = []
    total_score = 0
    iters = 1
    for i in range(iters):
        X_train, X_test, y_train, y_test = train_test_split(features, target)

        model = SVC(max_iter=10000).fit(X_train, y_train)

        actual = model.predict(X_test)
        expected = y_test.array
        correct = 0

        for index, age in enumerate(actual):
            if abs(age - expected[index] < 3):
                correct += 1
          

        train_acc = model.score(X_train, y_train)
        test_acc = model.score(X_test, y_test)

        total_score += test_acc

        data_rows.append({
            "train_acc": train_acc,
            "test_acc": test_acc,
            "our_test_acc": correct/len(X_test)
        })

        print(data_rows[i])

        a = model.predict([X_test.iloc[500]])
        e = expected[500]
        print(a)
        print(e) 

    test_df = pd.DataFrame(data_rows)
    print('average score', total_score/iters, '\n',test_df)

In [14]:
target = df['age']
df2 = df.drop(columns=['age'])

In [15]:
from sklearn.model_selection import train_test_split
features = df2

#print(features.head())
train_test_SVM(features, target)

{'train_acc': 0.4399290876858039, 'test_acc': 0.15132924335378323, 'our_test_acc': 0.8830265848670756}


IndexError: single positional indexer is out-of-bounds